In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
from copy import deepcopy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

from scipy import interp

from sklearn.metrics import roc_curve
from sklearn.preprocessing import LabelBinarizer

In [3]:
pip install scikit-optimize

     |████████████████████████████████| 81kB 5.4MB/s 


In [4]:
!git clone https://github.com/Qwicen/node.git

Cloning into 'node'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 33 (delta 8), reused 16 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [5]:
!pip install -r node/requirements.txt

     - 8.7MB 3.6MB/s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 55.5MB 51kB/s 
     |████████████████████████████████| 16.6MB 206kB/s 
     |████████████████████████████████| 317kB 53.6MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
  Created wheel for qhoptim: filename=qhoptim-1.1.0-cp36-none-any.whl size=20328 sha256=a80f9671f0bb2338eb4dff10c8958c85fa573070cbd91f63799d3ba2178f657e
  Stored in directory: /tmp/pip-ephem-wheel-cache-8dkisblp/wheels/63/53/2b/3b721f6feacd39063f6300e890b7cf633910a33e7b996edbd3
Successfully built qhoptim
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [6]:

%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from node import lib
import torch, torch.nn as nn
import torch.nn.functional as F
from qhoptim.pyt import QHAdam

from numpy import mean
from sklearn.datasets import make_blobs
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from skopt.space import Integer
from skopt.space import Real
from skopt.utils import use_named_args
from skopt import gp_minimize
import time


env: CUDA_VISIBLE_DEVICES=0,1


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Create a predict_proba function to extract probabilities from the model

In [8]:
def predict_proba(self, X_test, y_test, device, batch_size=32):
    X_test = torch.as_tensor(X_test, device=device)
    y_test = lib.utils.check_numpy(y_test)
    self.model.train(False)
    with torch.no_grad():
        logits = F.softmax(lib.utils.process_in_chunks(self.model, X_test, batch_size=batch_size), dim=1)
        logits = lib.utils.check_numpy(logits)
    return logits

In [9]:
lib.Trainer.predict_proba = predict_proba

In [10]:
files_path = '/path/to/datasets'

In [11]:
files_list = os.listdir(files_path)

In [12]:
def read_dataset(folder_path, filename):
    df = pd.read_csv(os.path.join(folder_path, filename))
    return df

In [13]:
def dummy_encode(df):
    cols_to_encode = list(df.select_dtypes(include=['category','object']))
    if len(cols_to_encode): 
        df = pd.get_dummies(df, columns = cols_to_encode, prefix=cols_to_encode)

    return df.values


In [14]:
def prepare_data(df, target_col=None):
    df = df.fillna(df.mean())
    if target_col:
        X = df.iloc[:, df.columns != target_col]
        y = df.loc[:, df.columns == target_col]
    else:
        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]
    lb = LabelEncoder()
    y = lb.fit_transform(y)
    X = dummy_encode(X)
    classes = np.unique(y)

    return X, y, classes


# Metrics calculations

In [15]:
def metrics_calc(y_true, y_pred, classes):

    y_true_label = label_binarize(y_true, classes)

    y_pred_arg = np.argmax(y_pred, axis=1)

    cnf_matrix = confusion_matrix(y_true, y_pred_arg)

    fp = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    fn = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    tp = np.diag(cnf_matrix)
    tn = cnf_matrix.sum() - (fp + fn + tp)

    fp = fp.astype(float)
    fn = fn.astype(float)
    tp = tp.astype(float)
    tn = tn.astype(float)

    tpr = tp/(tp+fn)
    fpr = fp/(fp+tn)
    precision = tp/(tp+fp)
    acc = (tp+tn)/(tp+fp+fn+tn)

    mean_acc = np.nanmean(acc)
    mean_tpr = np.nanmean(tpr)
    mean_fpr = np.nanmean(fpr)
    mean_precision = np.nanmean(precision)

    precision = dict()
    recall = dict()
    fpr = dict()
    tpr = dict()
    roc_auc_list = []
    pr_auc_list = []

    if len(classes) == 2:
        fpr, tpr, threshold_roc = roc_curve(y_true, y_pred[:, 1])

        roc_auc = auc(fpr, tpr)
        roc_auc_list.append(roc_auc)
        
        precision, recall, threshold_pr = precision_recall_curve(y_true,
                                                            y_pred[:, 1])
        
        pr_auc = auc(recall, precision)
        pr_auc_list.append(pr_auc)

    else:
        for i in range(y_pred.shape[1]):
            fpr[i], tpr[i], threshold_roc = roc_curve(y_true_label[:, i], y_pred[:, i])
            # roc_auc[i] = auc(fpr[i], tpr[i])
            roc_auc = auc(fpr[i], tpr[i])
            roc_auc_list.append(roc_auc)
            
            precision[i], recall[i], threshold_pr = precision_recall_curve(y_true_label[:, i],
                                                                y_pred[:, i])

            pr_auc = auc(recall[i], precision[i])
            pr_auc_list.append(pr_auc)

    mean_roc_auc = np.nanmean(roc_auc_list)
    mean_pr_auc = np.nanmean(pr_auc_list)

    print('mean roc auc',mean_roc_auc)
    print('mean pr auc', mean_pr_auc)

    return mean_acc, mean_tpr, mean_fpr, mean_precision, mean_roc_auc, mean_pr_auc


# Create model

In [16]:
def create_model(data, experiment_name, depth, num_layers):
    num_features = data.X_train.shape[1]
    num_classes = len(set(data.y_train) | set(data.y_test))

    model = nn.Sequential(
        lib.DenseBlock(num_features, layer_dim=128, num_layers=num_layers,
                    tree_dim=num_classes + 1, flatten_output=False,
                    depth=depth, choice_function=lib.entmax15, bin_function=lib.entmoid15),
        lib.Lambda(lambda x: x[..., :num_classes].mean(dim=-2)),
    ).to(device)

    model = model.double()

    with torch.no_grad():
        res = model(torch.as_tensor(data.X_train[:2000], device=device))
        
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)


    trainer = lib.Trainer(
        model=model, loss_function=F.cross_entropy,
        experiment_name=experiment_name,
        warm_start=False,
        Optimizer=QHAdam,
        optimizer_params=dict(nus=(0.7, 1.0), betas=(0.95, 0.998)),
        verbose=True,
        n_last_checkpoints=5
    )

    return trainer

#Hyperparameters optimization

In [17]:
def hyper_optimization(file, X, y):
    search_space = [Integer(2, 8 , name='depth', dtype=int),
                    Integer(2, 4, name='num_layers', dtype=int)]

    @use_named_args(search_space)
    def evaluate_model(**params):
        results = []
        kf = KFold(n_splits=3, random_state=42, shuffle=True)
        kf.get_n_splits(X)
        for train_index, test_index in kf.split(X, y):

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            X_, X_val, y_, y_val = train_test_split(X_train, y_train, 
                                                              test_size=0.1, shuffle=True, 
                                                              random_state=42)


            data = lib.Dataset(file, data_path=files_path,  X_train=X_train, y_train=y_train, 
                   X_valid=X_val, y_valid=y_val, X_test=X_test, y_test=y_test,
                   random_state=1337, quantile_transform=True)

            experiment_name = 'epsilon_node.{}'.format(time.time())
            device = 'cuda' if torch.cuda.is_available() else 'cpu'

            model = create_model(data, experiment_name, **params)

            for batch in lib.iterate_minibatches(data.X_train, data.y_train, batch_size=32, 
                                                shuffle=True, epochs=float(5)):
                
                metrics = model.train_on_batch(*batch, device=device)
            
            y_pred = model.predict_proba(data.X_test, data.y_test,
                                         device=device, batch_size=32)
            y_pred_arg = np.argmax(y_pred, axis=1)

            result = accuracy_score(y_test, y_pred_arg)

            results.append(result)

        # calculate the mean of the scores
        estimate = mean(results)
        print('mean resutls', estimate)
        return 1.0 - estimate

    # perform optimization
    result = gp_minimize(evaluate_model, search_space, n_calls=50)

    return result


# Training

In [ ]:
for file in files_list:
    print('file', file)

    df = read_dataset(files_path, file)

    if file == 'analcatdata_germangss.csv':
        X, y, classes = prepare_data(df, 'Political_system')
    else:
        X, y, classes = prepare_data(df)

    result = hyper_optimization(file, X, y)

    best_params_vals = result.x
    params_name = ['depth', 'num_layers']
    best_params = dict(zip(params_name, best_params_vals))

    print(best_params)

    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    kf.get_n_splits(X)

    acc_list = []
    tpr_list = []
    fpr_list = []
    precision_list = []
    roc_auc_list = []
    pr_auc_list = []
    training_time_list = []
    inference_time_list = []
    cross_val = []

    for fold_num, data_index in enumerate(kf.split(X, y)):
        train_index, test_index = data_index[0], data_index[1]
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_, X_val, y_, y_val = train_test_split(X_train, y_train,
                                                          test_size=0.1,
                                                          shuffle=True,
                                                          random_state=42)


        data = lib.Dataset(file, data_path=files_path,  X_train=X_train, y_train=y_train, 
                X_valid=X_val, y_valid=y_val, X_test=X_test, y_test=y_test,
                random_state=1337, quantile_transform=True)
        experiment_name = 'epsilon_node.{}'.format(time.time())

        model = create_model(data, experiment_name, **best_params)

        t0 = time.time()

        for batch in lib.iterate_minibatches(data.X_train, data.y_train,
                                             batch_size=32, shuffle=True,
                                             epochs=float(20)
                                             ):
            metrics = model.train_on_batch(*batch, device=device)

        t1 = time.time()
        train_time = t1 - t0
        print('train time', train_time)

        t2 = time.time()
        y_pred = model.predict_proba(data.X_test, data.y_test,
                                        device=device, batch_size=128)
        t3 = time.time()
        inference_time = (t3 - t2) * 1000 / y_test.shape[0]

        acc, mean_tpr, mean_fpr, mean_precision, mean_roc_auc, mean_pr_auc = metrics_calc(y_test, y_pred, classes)

        acc_list.append("{:.3f}".format(acc))
        tpr_list.append("{:.3f}".format(mean_tpr))
        fpr_list.append("{:.3f}".format(mean_fpr))
        precision_list.append("{:.3f}".format(mean_precision))
        roc_auc_list.append("{:.3f}".format(mean_roc_auc))
        pr_auc_list.append("{:.3f}".format(mean_pr_auc))
        training_time_list.append("{:.1f}".format(train_time))
        inference_time_list.append("{:.1f}".format(inference_time))

        cross_val.append(fold_num + 1)

    results_dict = {'Dataset Name':[file.split('.')[0]] * 10,
                'Algorithm Name':['node'] * 10,
                'Cross Validation':cross_val,
                'Hyper Parameters Values': [best_params_vals] * 10,
                'Accuracy':acc_list,
                'tpr':tpr_list,
                'FPR':fpr_list,
                'Precision':precision_list,
                'AUC':roc_auc_list,
                'PR-Curve':pr_auc_list,
                'Training Time':training_time_list,
                'Inference Time':inference_time_list
                }

    df_res = pd.DataFrame.from_dict(results_dict)
    df_res = df_res.fillna(0)

    df_res.to_csv('path/to/results',
              mode='a', header=False)
    
    model.zero_grad()
    torch.cuda.empty_cache()